## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-05-12-59-56 +0000,nyt,Syria Chooses Its First Parliament Since End o...,https://www.nytimes.com/2025/10/05/world/middl...
1,2025-10-05-12-56-53 +0000,bbc,Al-Shabab militants dress as soldiers to storm...,https://www.bbc.com/news/articles/cp3v0px28j8o...
2,2025-10-05-12-55-00 +0000,wsj,OPEC and Allies Agree to Boost Oil Production,https://www.wsj.com/business/energy-oil/opec-a...
3,2025-10-05-12-44-39 +0000,nyt,OPEC Plus Agrees to Small Boost in Oil Production,https://www.nytimes.com/2025/10/05/business/op...
4,2025-10-05-12-25-51 +0000,bbc,"My approach will pay off eventually, says Kemi...",https://www.bbc.com/news/articles/c1l81766g2qo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2357/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
63,trump,32
112,gaza,18
108,israel,11
107,plan,11
193,deal,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
84,2025-10-05-02-19-00 +0000,wsj,President Trump pressed ahead with plans to mo...,https://www.wsj.com/politics/policy/trump-auth...,116
124,2025-10-04-20-36-56 +0000,nypost,Israel agrees to ‘initial withdrawal line’ in ...,https://nypost.com/2025/10/04/world-news/israe...,103
15,2025-10-05-11-51-27 +0000,nyt,Israel and Hamas Prepare for Talks on Trump’s ...,https://www.nytimes.com/2025/10/05/world/middl...,100
179,2025-10-04-14-18-43 +0000,nypost,Hamas’ response to Trump’s Gaza peace plan win...,https://nypost.com/2025/10/04/world-news/hamas...,98
183,2025-10-04-13-57-38 +0000,bbc,Shock in Gaza as Trump appears to welcome Hama...,https://www.bbc.com/news/articles/c15k199j1x3o...,85


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
84,116,2025-10-05-02-19-00 +0000,wsj,President Trump pressed ahead with plans to mo...,https://www.wsj.com/politics/policy/trump-auth...
124,71,2025-10-04-20-36-56 +0000,nypost,Israel agrees to ‘initial withdrawal line’ in ...,https://nypost.com/2025/10/04/world-news/israe...
159,44,2025-10-04-16-28-25 +0000,nypost,London police arrest dozens of protesters agai...,https://nypost.com/2025/10/04/world-news/londo...
184,35,2025-10-04-13-48-43 +0000,nypost,Israel’s army to ‘advance readiness’ for first...,https://nypost.com/2025/10/04/world-news/israe...
101,32,2025-10-04-23-14-00 +0000,wsj,Defense Secretary Pete Hegseth’s emphasis on p...,https://www.wsj.com/politics/national-security...
102,31,2025-10-04-23-05-09 +0000,nypost,Youngkin calls for embattled Dem AG hopeful Ja...,https://nypost.com/2025/10/04/us-news/virginia...
127,25,2025-10-04-20-11-06 +0000,nyt,Andrej Babis Likely to Return as Czech Prime M...,https://www.nytimes.com/2025/10/04/world/europ...
162,25,2025-10-04-16-00-00 +0000,wsj,"To hear the top House Democrat tell it, beyond...",https://www.wsj.com/politics/policy/hakeem-jef...
186,24,2025-10-04-13-20-34 +0000,bbc,French photojournalist killed in drone strike ...,https://www.bbc.com/news/articles/crl57n318ezo...
114,22,2025-10-04-21-18-46 +0000,nypost,Former Syrian ruler Bashar al-Assad nearly die...,https://nypost.com/2025/10/04/world-news/ouste...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
